In [1]:
#!pip install datasets
#!pip install transformers


from datasets import load_dataset
from torch.utils.data import DataLoader
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
import copy
from tqdm import tqdm_notebook
import os
from torch.optim.lr_scheduler import ReduceLROnPlateau
import warnings

warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# сюда нужно поместить ваш токен который дает доступ к гиту, его можно сделать в настройках гита
!git clone https://<token>@github.com/EkaterinaAdishcheva/LinAlg_Project_AI_Master.git

Cloning into 'LinAlg_Project_AI_Master'...
remote: Enumerating objects: 2793, done.
remote: Counting objects: 100% (2793/2793), done.
remote: Compressing objects: 100% (1811/1811), done.
remote: Total 2793 (delta 1145), reused 2608 (delta 980), pack-reused 0
Receiving objects: 100% (2793/2793), 5.47 MiB | 5.55 MiB/s, done.
Resolving deltas: 100% (1145/1145), done.


In [2]:
from LinAlg_Project_AI_Master.src.transformers_my.models.bert.modeling_bert import (
    BertForMaskedLM,
)
from LinAlg_Project_AI_Master.src.transformers_my.models.bert.tokenization_bert import (
    BertTokenizer,
)

In [3]:
BertForMaskedLM

LinAlg_Project_AI_Master.src.transformers_my.models.bert.modeling_bert.BertForMaskedLM

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased", return_dict=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
cola = load_dataset("glue", "cola", split="train")

In [6]:
class MaskedCola(object):
    def __init__(self, dataset, tokenizer):
        self.df = dataset
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        # text = self.df[idx]['sentence'].lower()
        text = self.df["sentence"][idx].lower()

        inputs = self.tokenizer.encode_plus(
            text,  # Sentence to encode.
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=64,  # Pad & truncate all sentences.
            pad_to_max_length=True,
            return_attention_mask=True,  # Construct attn. masks.
            return_tensors="pt",  # Return pytorch tensors.
            truncation=True,  # Truncate everything more than max len
        )
        input_ids = inputs["input_ids"]

        labels = copy.deepcopy(input_ids)
        masked_idx = np.random.randint(sum(input_ids[0] != 0))

        input_ids[0][masked_idx] = tokenizer.mask_token_id

        labels[input_ids != tokenizer.mask_token_id] = -100

        # input_ids = input_ids, attention_mask = inputs['attention_mask'] , token_type_ids=inputs['token_type_ids'] , labels=labels

        return {
            "input_ids": input_ids,
            "attention_mask": inputs["attention_mask"],
            "token_type_ids": inputs["token_type_ids"],
            "labels": labels,
        }

    def __len__(self):
        return len(self.df["sentence"])


def collate_fn(batch):
    iid_ls = []
    atm_ls = []
    tti_ls = []
    l_ls = []

    for elem in batch:
        iid = elem["input_ids"]
        atm = elem["attention_mask"]
        tti = elem["token_type_ids"]
        l = elem["labels"]

        iid_ls.append(iid)
        atm_ls.append(atm)
        tti_ls.append(tti)
        l_ls.append(l)

    return torch.cat(iid_ls), torch.cat(atm_ls), torch.cat(tti_ls), torch.cat(l_ls)

In [7]:
dataset_train = MaskedCola(cola[:7000], tokenizer)
dataset_val = MaskedCola(cola[7000:], tokenizer)

dataloader_train = DataLoader(
    dataset_train,
    batch_size=128,
    shuffle=False,
    sampler=None,
    collate_fn=collate_fn,
    drop_last=False,
)
dataloader_val = DataLoader(
    dataset_val,
    batch_size=128,
    shuffle=False,
    sampler=None,
    collate_fn=collate_fn,
    drop_last=False,
)

In [16]:
# Функции для сохранения и загрузки чекпоинтов
def save_checkpoint(model, filename):
    with open(filename, "wb") as fp:
        torch.save(model.state_dict(), fp)


def load_checkpoint(model, filename):
    with open(filename, "rb") as fp:
        state_dict = torch.load(fp)

    model.load_state_dict(state_dict)


# цикл для трейна
def run_epoch_train(
    model, dataloader, loss_fn, optimizer, epoch, device, scheduler=None, log_acc=False
):
    stage = "train"

    model.train()
    torch.set_grad_enabled(True)

    model = model.to(device)

    if log_acc:
        total_true = 0
        total_labels = 0

    losses = []
    for batch in tqdm_notebook(
        dataloader,
        total=len(dataloader),
        desc=f"epoch: {str(epoch).zfill(3)} | {stage:5}",
    ):
        inp_ids = batch[0].to(device)
        att_masks = batch[1].to(device)
        token_types = batch[2].to(device)
        labels = batch[3].to(device)

        out = model(
            input_ids=inp_ids,
            attention_mask=att_masks,
            token_type_ids=token_types,
            labels=labels,
        )

        loss = out["loss"]
        logits = out["logits"]

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        losses.append(loss.detach().cpu().item())

        if log_acc:
            mask_token_index = (inp_ids == tokenizer.mask_token_id).nonzero(
                as_tuple=True
            )[1]

            pred_tokens = logits[:, mask_token_index, :].argmax(axis=-1).diag()

            true_preds = labels[labels != -100] == pred_tokens

            total_true += true_preds.sum().item()
            total_labels += len(true_preds)

    if scheduler is not None:
        scheduler.step(np.mean(losses))

    if log_acc:
        acc = total_true / total_labels
        print(acc)

    return np.mean(losses)


# цикл для валидации
def run_epoch_val(
    model, dataloader, loss_fn, optimizer, epoch, device, scheduler=None, log_acc=False
):
    stage = "val"
    torch.set_grad_enabled(False)
    model.eval()

    if log_acc:
        total_true = 0
        total_labels = 0

    model = model.to(device)

    losses = []
    for batch in tqdm_notebook(
        dataloader,
        total=len(dataloader),
        desc=f"epoch: {str(epoch).zfill(3)} | {stage:5}",
    ):
        inp_ids = batch[0].to(device)
        att_masks = batch[1].to(device)
        token_types = batch[2].to(device)
        labels = batch[3].to(device)

        out = model(
            input_ids=inp_ids,
            attention_mask=att_masks,
            token_type_ids=token_types,
            labels=labels,
        )

        loss = out["loss"]
        logits = out["logits"]

        losses.append(loss.detach().cpu().item())

        if log_acc:
            mask_token_index = (inp_ids == tokenizer.mask_token_id).nonzero(
                as_tuple=True
            )[1]

            pred_tokens = logits[:, mask_token_index, :].argmax(axis=-1).diag()

            true_preds = labels[labels != -100] == pred_tokens

            total_true += true_preds.sum().item()
            total_labels += len(true_preds)

    if log_acc:
        acc = total_true / total_labels
        print(acc)

    return np.mean(losses)


# цикл для прохождения по эпохам, обучения, валидации и логирования
def run_experiment(
    model,
    dataloader_train,
    dataloader_val,
    loss_fn,
    optimizer,
    num_epochs,
    device,
    output_dir,
    scheduler,
    log_acc=True,
):
    train_losses = []
    val_losses = []

    best_val_loss = np.inf
    best_val_loss_epoch = -1
    best_val_loss_fn = None

    os.makedirs(output_dir, exist_ok=True)

    for epoch in range(num_epochs):
        train_loss = run_epoch_train(
            model,
            dataloader_train,
            loss_fn,
            optimizer,
            epoch,
            device,
            scheduler,
            log_acc=log_acc,
        )
        train_losses.append(train_loss)

        val_loss = run_epoch_val(
            model, dataloader_val, loss_fn, optimizer, epoch, device, log_acc=log_acc
        )
        val_losses.append(val_loss)

        print(
            f"epoch: {str(epoch).zfill(3)} | train_loss: {train_loss:5.3f}, val_loss: {val_loss:5.3f} (best: {best_val_loss:5.3f})"
        )

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_loss_epoch = epoch

            output_fn = os.path.join(
                output_dir,
                f"epoch={str(epoch).zfill(2)}_valloss={best_val_loss:.3f}.pth.tar",
            )
            save_checkpoint(model, output_fn)
            print(f"New checkpoint saved to {output_fn}")

            # best_val_loss_fn = output_fn

        print()

    print(f"Best val_loss = {best_val_loss:.3f} reached at epoch {best_val_loss_epoch}")
    # load_checkpoint(model, best_val_loss_fn)

    return train_losses, val_losses, best_val_loss, model


def validate(model, dataloader, device):
    model.eval()
    model = model.to(device)

    total_true = 0
    total_labels = 0

    for batch in tqdm_notebook(dataloader):
        inp_ids = batch[0].to(device)
        att_masks = batch[1].to(device)
        token_types = batch[2].to(device)
        labels = batch[3].to(device)

        out = model(
            input_ids=inp_ids,
            attention_mask=att_masks,
            token_type_ids=token_types,
            labels=labels,
        )
        logits = out["logits"]

        mask_token_index = (inp_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[
            1
        ]

        pred_tokens = logits[:, mask_token_index, :].argmax(axis=-1).diag()

        true_preds = labels[labels != -100] == pred_tokens

        total_true += true_preds.sum().item()
        total_labels += len(true_preds)

    return total_true / total_labels


def truncation_finetuning(
    model,
    dataloader_train,
    dataloader_val,
    lr,
    num_epochs,
    device,
    output_dir,
    compression_scheme,
):
    encoder_layer_size = 768

    r_ls = [
        int(encoder_layer_size * 1 / compress_ratio)
        for compress_ratio in compression_scheme
    ]

    acc_ls = []

    model.to(device)

    for compression_stage in range(len(compression_scheme)):
        # compressing
        r = r_ls[compression_stage]
        model.set_truncation(r)
        print(r)

        # freeze everything except embedding params
        """
        for param in model.parameters():
          #print(param)
          param.requires_grad = False

        for param in model.bert.embeddings.parameters():
          param.requires_grad = True
        """

        # defining params
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        scheduler = ReduceLROnPlateau(
            optimizer, mode="min", factor=0.1, patience=4, verbose=True, min_lr=3e-6
        )

        # fintuning
        (
            train_losses_baseline,
            val_losses_baseline,
            best_val_loss_baseline,
            cnn_baseline,
        ) = run_experiment(
            model,
            dataloader_train,
            dataloader_val,
            loss_fn,
            optimizer,
            num_epochs,
            device,
            output_dir,
            scheduler,
        )

        # validation + accuracy measure

        acc = validate(model, dataloader_val, device)

        acc_ls.append(acc)

        # decompressing
        W = (
            model.bert.embeddings.state_dict()["U"]
            * model.bert.embeddings.state_dict()["S"]
        ) @ model.bert.embeddings.state_dict()["Vt"]
        model.bert.embeddings.word_embeddings.weight.data = W

    return acc_ls

In [20]:
lr = 3e-5
num_epochs = 15

compression_scheme = [2, 4, 8, 10, 12]
output_dir = "Iter_compress_no_freez_2_4_8_10_12"

In [ ]:
acc_ls = truncation_finetuning(
    model,
    dataloader_train,
    dataloader_val,
    lr,
    num_epochs,
    device,
    output_dir,
    compression_scheme,
)

Embeddings Truncated
384


epoch: 000 | train:   0%|          | 0/55 [00:00<?, ?it/s]

0.679


epoch: 000 | val  :   0%|          | 0/13 [00:00<?, ?it/s]

0.4261766602192134
epoch: 000 | train_loss: 1.643, val_loss: 4.829 (best:   inf)
New checkpoint saved to Iter_compress_no_freez_2_4_8_10_12/epoch=00_valloss=4.829.pth.tar



...

epoch: 008 | val  :   0%|          | 0/13 [00:00<?, ?it/s]

0.44294003868471954
epoch: 008 | train_loss: 1.511, val_loss: 4.794 (best: 4.544)



epoch: 009 | train:   0%|          | 0/55 [00:00<?, ?it/s]

0.6891428571428572


epoch: 009 | val  :   0%|          | 0/13 [00:00<?, ?it/s]